In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.5 MB/s 
     |████████████████████████████████| 7.6 MB 52.1 MB/s 
     |████████████████████████████████| 163 kB 76.9 MB/s 


In [2]:
# !pip install datasets

In [3]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py

# 모듈 import 및 전역 변수 설정

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# cd /content/drive/MyDrive/python_codes/NLP/py-hanspell-master

In [6]:
# !python setup.py install

In [7]:
# from hanspell import spell_checker

In [8]:
# !sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

In [9]:
# !python3 -m pip install konlpy 

In [10]:
# from konlpy.tag import Okt
# okt = Okt()

In [11]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
# from transformers import XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import copy

import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import pickle

for_test = True

PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

do_eval=True
FULL_FINETUNING = True
category_extraction_model_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/category_extraction'
polarity_classification_model_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/polarity_classification'

train_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-train.jsonl'
dev_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-dev.jsonl'
test_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-test.jsonl'

max_len = 256
batch_size = 8
base_model = 'xlm-roberta-base'
base_model = 'klue/roberta-large'
entity_property_learning_rate = 3e-6
polarity_learning_rate = 3e-6
eps = 1e-8
num_train_epochs = 100
classifier_hidden_size = 768
classifier_dropout_prob = 0.1

entity_property_patience_ub = 10
polarity_patience_ub = 10
val_num = 5

entity_property_pair = ['본품#품질',
'제품 전체#일반',
'제품 전체#품질',
'본품#일반',
'제품 전체#디자인',
'본품#편의성',
'제품 전체#편의성',
'제품 전체#인지도',
'패키지/구성품#디자인',
'브랜드#일반',
'제품 전체#가격',
'패키지/구성품#편의성',
'패키지/구성품#일반',
# '본품#다양성',
# '본품#디자인',
# '브랜드#품질',
# '패키지/구성품#품질',
# '브랜드#인지도',
# '브랜드#가격',
# '패키지/구성품#다양성',
# '본품#가격',
# '본품#인지도',
# '패키지/구성품#가격'
]

tf_id_to_name = ['True', 'False']
tf_id_to_name = ['본품#품질',
'제품 전체#일반',
'제품 전체#품질',
'본품#일반',
'제품 전체#디자인',
'본품#편의성',
'제품 전체#편의성',
'제품 전체#인지도',
'패키지/구성품#디자인',
'브랜드#일반',
'제품 전체#가격',
'패키지/구성품#편의성',
'패키지/구성품#일반',
# '본품#다양성',
# '본품#디자인',
# '브랜드#품질',
# '패키지/구성품#품질',
# '브랜드#인지도',
# '브랜드#가격',
# '패키지/구성품#다양성',
# '본품#가격',
# '본품#인지도',
# '패키지/구성품#가격'
]
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'neutral', 'negative']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

json 및 jsonl 파일 read, write 함수

In [12]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list


# jsonlload('/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/sample.jsonl')

In [13]:
tmp_list = jsonlload(train_data_path)
tmp_list.extend(jsonlload(dev_data_path))
tmp_df = pd.DataFrame()
k=0
for i in range(len(tmp_list)):
  for j in range(len(tmp_list[i]['annotation'])):
    tmp_df.loc[k,'entity_property'] = tmp_list[i]['annotation'][j][0]
    tmp_df.loc[k,'polarity'] = tmp_list[i]['annotation'][j][-1]
    k+=1
tmp_df

,entity_property,polarity
0,본품#품질,negative
1,본품#품질,negative
2,제품 전체#일반,positive
3,제품 전체#일반,neutral
4,본품#일반,negative
...,...,...
6329,패키지/구성품#일반,positive
6330,제품 전체#가격,negative
6331,제품 전체#가격,negative
6332,제품 전체#일반,positive


In [14]:
tmp_df['entity_property'].value_counts()

본품#품질          2380
제품 전체#일반       1624
제품 전체#품질        493
본품#일반           491
제품 전체#디자인       286
본품#편의성          191
제품 전체#편의성       180
제품 전체#인지도       141
패키지/구성품#디자인     117
브랜드#일반          103
제품 전체#가격         92
패키지/구성품#편의성      65
패키지/구성품#일반       50
본품#다양성           31
본품#디자인           21
브랜드#품질           19
패키지/구성품#품질       19
브랜드#인지도          17
브랜드#가격            7
패키지/구성품#다양성       3
본품#가격             2
본품#인지도            1
패키지/구성품#가격        1
Name: entity_property, dtype: int64

In [15]:
import numpy as np
class_weight = list(len(tmp_df['entity_property'])/(tmp_df['entity_property'].value_counts()+100)) # entity_property의 각각 label의 positive에 대해 줄 가중치샘플비율대로 하면 너무 심해서 로그씌움.
class_weight

[2.5540322580645163,
 3.6740139211136893,
 10.681281618887015,
 10.717428087986464,
 16.409326424870468,
 21.766323024054984,
 22.62142857142857,
 26.282157676348547,
 29.1889400921659,
 31.201970443349754,
 32.989583333333336,
 38.38787878787879,
 42.22666666666667,
 48.35114503816794,
 52.34710743801653,
 53.226890756302524,
 53.226890756302524,
 54.136752136752136,
 59.19626168224299,
 61.49514563106796,
 62.09803921568628,
 62.71287128712871,
 62.71287128712871]

In [16]:
import numpy as np
class_weight = list(len(tmp_df['entity_property'])/(tmp_df['entity_property'].value_counts()+100)) # entity_property의 각각 label의 positive에 대해 줄 가중치샘플비율대로 하면 너무 심해서 로그씌움.
class_weight = class_weight[:-10]
class_weight

[2.5540322580645163,
 3.6740139211136893,
 10.681281618887015,
 10.717428087986464,
 16.409326424870468,
 21.766323024054984,
 22.62142857142857,
 26.282157676348547,
 29.1889400921659,
 31.201970443349754,
 32.989583333333336,
 38.38787878787879,
 42.22666666666667]

In [17]:
polarity_class_weight = list(len(tmp_df['polarity'])/tmp_df['polarity'].value_counts())
polarity_class_weight

[1.0385309067060173, 42.51006711409396, 73.65116279069767]

In [18]:
polarity_class_weight = list(np.sqrt(len(tmp_df['polarity'])/(tmp_df['polarity'].value_counts())))
polarity_class_weight

[1.0190833659254857, 6.5199744718897445, 8.582025564556288]

In [19]:
import pandas as pd
pd.to_datetime('2013.1')

Timestamp('2013-01-01 00:00:00')

# 모델 정의
xlm-roberta 모델을 기반으로 한 classification 모델 이용

In [20]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        # self.dense = nn.Linear(classifier_hidden_size, classifier_hidden_size)
        self.dense = nn.Linear(1024, 50)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(50, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x


class RoBertaBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RoBertaBaseClassifier, self).__init__()

        self.num_label = num_label
        self.xlm_roberta = AutoModel.from_pretrained(base_model)
        for param in self.xlm_roberta.parameters():
            param.requires_grad = False

        self.xlm_roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.xlm_roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        # logits = torch.sigmoid(logits)

        loss = None

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss(
                pos_weight=torch.tensor(class_weight).to(device)
                )
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels)
        # logits = torch.sigmoid(logits)

        return loss, logits

class RoBertaBaseClassifier_polar(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RoBertaBaseClassifier_polar, self).__init__()

        self.num_label = num_label
        self.xlm_roberta = AutoModel.from_pretrained(base_model)
        for param in self.xlm_roberta.parameters():
            param.requires_grad = False

        self.xlm_roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.xlm_roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(
                weight=torch.tensor(polarity_class_weight).to(device)
                )
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


# 데이터 파싱 및 tokenization 함수 정의


In [21]:

def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    polarity_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    tokenized_data = tokenizer(form, '미정', padding='max_length', max_length=max_len, truncation=True)
    entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
    entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
    entity_property_data_dict['label'].append([0.0]*13)
    isPairInOpinion = False
    for pair in entity_property_pair:
        
        if pd.isna(form):
            break
        

        for annotation in annotations:
            entity_property = annotation[0]
            polarity = annotation[2]

            if polarity == '------------':
                continue

            if entity_property == pair:
                entity_property_data_dict['label'][-1][tf_name_to_id[entity_property]] = 1.0

                tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
                polarity_data_dict['input_ids'].append(tokenized_data['input_ids'])
                polarity_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                polarity_data_dict['label'].append(polarity_name_to_id[polarity])

                # isPairInOpinion = True
                break


    return entity_property_data_dict, polarity_data_dict


def get_dataset(raw_data, tokenizer, max_len):
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []

    polarity_input_ids_list = []
    polarity_attention_mask_list = []
    polarity_token_labels_list = []

    for utterance in raw_data:
        entity_property_data_dict, polarity_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])

        polarity_input_ids_list.extend(polarity_data_dict['input_ids'])
        polarity_attention_mask_list.extend(polarity_data_dict['attention_mask'])
        polarity_token_labels_list.extend(polarity_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list),
                         torch.tensor(token_labels_list)), TensorDataset(torch.tensor(polarity_input_ids_list), torch.tensor(polarity_attention_mask_list),
                         torch.tensor(polarity_token_labels_list))



# 모델 평가

학습된 모델을 바탕으로 국어원 데이터 형태를 만드는 방법 예시

In [22]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def find_threshold(metric = 'f1'):
  tokenizer = AutoTokenizer.from_pretrained(base_model)
  num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
  thresholds = {}
  # with open('/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/concatenated_preproced_data.pkl', 'rb') as f:
  #     train_data = pickle.load(f)
  train_data = jsonlload(train_data_path)
  train_data.extend(jsonlload(dev_data_path))
  tmp_df = pd.DataFrame()
  tmp_train = []
  k=0
  for i in range(len(train_data)):
    if train_data[i]['annotation'][0][0] in entity_property_pair:
      tmp_train.append(train_data[i])
      tmp_df.loc[k,'entity_property'] = train_data[i]['annotation'][0][0]
      k += 1
  from sklearn.model_selection import StratifiedKFold
  skf = StratifiedKFold(n_splits=5, random_state = 0, shuffle=True)
  val_step = 0
  for train_index, test_index in skf.split(tmp_train, tmp_df['entity_property']):
    print('\n\n======================= {} step=========================\n\n\n\n\n\n\n\n'.format(val_step+1))
    thresholds[val_step] = {}
    tmp_dev_data = []
    tmp_train_data = []
    for i in test_index:
      tmp_dev_data.append(tmp_train[i])

    test_category_extraction_model_path = category_extraction_model_path+'saved_model_epoch_{}.pt'.format(val_step)
    model = RoBertaBaseClassifier(len(tf_id_to_name), len(tokenizer))
    model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    model.to(device)

    tmp_input_ids = []
    tmp_attention_masks = []
    tmp_labels = []
    for j in range(len(tmp_dev_data)):
      tokenized_data = tokenizer(tmp_dev_data[j]['sentence_form'], '미정', padding='max_length', max_length=max_len, truncation=True)
      tmp_input_ids.append(tokenized_data['input_ids'])
      tmp_attention_masks.append(tokenized_data['attention_mask'])
      tmp_labels.append([0.0]*13)

      for i in range(len(tmp_dev_data[j]['annotation'])):
        if tmp_dev_data[j]['annotation'][i][0] in entity_property_pair:
          tmp_labels[-1][tf_name_to_id[tmp_dev_data[j]['annotation'][i][0]]] = 1
    input_ids = torch.tensor(tmp_input_ids).to(device)
    attention_masks = torch.tensor(tmp_attention_masks).to(device)

    with torch.no_grad():
      _, ce_logits = model(input_ids, attention_masks, None)
    y_true = pd.DataFrame(np.asarray(tmp_labels), columns = tf_id_to_name)
    y_pred = pd.DataFrame(np.asarray(ce_logits.sigmoid()), columns = tf_id_to_name)
    y_pred2 = y_pred.copy()
    for i in range(len(list(y_true))):
      thresholds[val_step][tf_id_to_name[i]] = []
      for j in range(30, 95, 1):
        threshold = j/100
        if y_true[tf_id_to_name[i]].sum()!=0:
          if metric == 'f1':
            thresholds[val_step][tf_id_to_name[i]].append(f1_score(y_true[tf_id_to_name[i]], y_pred[tf_id_to_name[i]] > threshold))
          elif metric == 'accuracy':
            thresholds[val_step][tf_id_to_name[i]].append(accuracy_score(y_true[tf_id_to_name[i]], y_pred[tf_id_to_name[i]] > threshold))

        else:
          thresholds[val_step][tf_id_to_name[i]].append(1)
      print('val_step {}, entity {} best threshold: {} f1 score: {}'.format(val_step, tf_id_to_name[i], np.arange(30, 95, 1)[np.argmax(thresholds[val_step][tf_id_to_name[i]])], np.max(thresholds[val_step][tf_id_to_name[i]])))
      # for i in range(len(list(y_true))):
      #   y_pred2[tf_id_to_name[i]] = y_pred2[tf_id_to_name[i]] > np.arange(30, 95, 1)[np.argmax(thresholds[val_step][tf_id_to_name[i]])]/100
    val_step += 1

  return thresholds

In [ ]:
f1_scores  = find_threshold(
    # metric = 'accuracy'
    )

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]



======================= 1 step=========================










...

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

val_step 0, entity 본품#품질 best threshold: 64 f1 score: 0.653256704980843
val_step 0, entity 제품 전체#일반 best threshold: 59 f1 score: 0.5111940298507464
val_step 0, entity 제품 전체#품질 best threshold: 47 f1 score: 0.1678756476683938
val_step 0, entity 본품#일반 best threshold: 50 f1 score: 0.15942028985507245
val_step 0, entity 제품 전체#디자인 best threshold: 48 f1 score: 0.21428571428571427
val_step 0, entity 본품#편의성 best threshold: 39 f1 score: 0.08875739644970415
val_step 0, entity 제품 전체#편의성 best threshold: 44 f1 score: 0.125
val_step 0, entity 제품 전체#인지도 best threshold: 38 f1 score: 0.10309278350515463
val_step 0, entity 패키지/구성품#디자인 best threshold: 38 f1 score: 0.06666666666666667
val_step 0, entity 브랜드#일반 best threshold: 32 f1 score: 0.0900900900900901
val_step 0, entity 제품 전체#가격 best threshold: 33 f1 score: 0.04761904761904762
val_step 0, entity 패키지/구성품#편의성 best threshold: 30 f1 score: 0.0
val_step 0, entity 패키지/구성품#일반 best threshold: 30 f1 score: 0.0


======================= 2 step=================

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

val_step 1, entity 본품#품질 best threshold: 64 f1 score: 0.8078175895765471
val_step 1, entity 제품 전체#일반 best threshold: 65 f1 score: 0.7566666666666667
val_step 1, entity 제품 전체#품질 best threshold: 66 f1 score: 0.35748792270531393
val_step 1, entity 본품#일반 best threshold: 80 f1 score: 0.7431693989071039
val_step 1, entity 제품 전체#디자인 best threshold: 67 f1 score: 0.6299212598425197
val_step 1, entity 본품#편의성 best threshold: 74 f1 score: 0.48648648648648646
val_step 1, entity 제품 전체#편의성 best threshold: 82 f1 score: 0.4897959183673469
val_step 1, entity 제품 전체#인지도 best threshold: 58 f1 score: 0.793103448275862
val_step 1, entity 패키지/구성품#디자인 best threshold: 87 f1 score: 0.761904761904762
val_step 1, entity 브랜드#일반 best threshold: 89 f1 score: 0.6666666666666665
val_step 1, entity 제품 전체#가격 best threshold: 91 f1 score: 0.85
val_step 1, entity 패키지/구성품#편의성 best threshold: 81 f1 score: 0.5294117647058824
val_step 1, entity 패키지/구성품#일반 best threshold: 73 f1 score: 0.3636363636363636


=======================

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [ ]:
import pickle
with open(category_extraction_model_path +'f1_scores.pkl', 'wb') as f:
  pickle.dump(f1_scores, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def predict_entity_property(test_data):
  import numpy as np
  tokenizer = AutoTokenizer.from_pretrained(base_model)
  num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
  logits = []
  for i in range(5):
    print(i)
    test_category_extraction_model_path = category_extraction_model_path+'saved_model_epoch_{}.pt'.format(i)
    model = RoBertaBaseClassifier(len(tf_id_to_name), len(tokenizer))
    model.load_state_dict(torch.load(test_category_extraction_model_path, map_location=device))
    model.to(device)


    tmp_input_ids = []
    tmp_attention_masks = []
    tmp_labels = []
    for j in range(len(test_data)):
      tokenized_data = tokenizer(test_data[j]['sentence_form'], '미정', padding='max_length', max_length=max_len, truncation=True)
      tmp_input_ids.append(tokenized_data['input_ids'])
      tmp_attention_masks.append(tokenized_data['attention_mask'])
      # tmp_labels.append(np.asarray([0.0]*23).reshape(1,23))

    input_ids = torch.tensor(tmp_input_ids).to(device)
    attention_masks = torch.tensor(tmp_attention_masks).to(device)
    # labels = torch.tensor(tmp_labels).to(device)

    with torch.no_grad():
      _, ce_logits = model(input_ids, attention_masks, None)
    logits.append(ce_logits)
  return logits

In [ ]:
test_data = jsonlload(test_data_path)
logits = predict_entity_property(test_data)

In [ ]:

import pickle
with open(category_extraction_model_path +'entity_property_logits.pkl', 'wb') as f:
  pickle.dump(logits, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open(category_extraction_model_path+'entity_property_logits.pkl', 'rb') as f:
    entity_property_logits = pickle.load(f)
entity_property_logits

In [ ]:
import copy
def predict_polarity_vote(test_data, entity_property_logits, f1_scores):
  tokenizer = AutoTokenizer.from_pretrained(base_model)
  num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
  entity_property_logits_mean = entity_property_logits[0]+entity_property_logits[1]+entity_property_logits[2]+entity_property_logits[3]+entity_property_logits[4]
  entity_property_logits_mean = np.asarray((entity_property_logits_mean/5).sigmoid())

  tmp_df = []
  for i in range(len(f1_scores)):
    tmp_df.append(pd.DataFrame(np.asarray(entity_property_logits[i].sigmoid()), columns = entity_property_pair))
    for j in list(f1_scores[i]):
      f1_scores[i][j]
      threshold = np.arange(30, 95, 1)[np.argmax(f1_scores[i][j])]/100
      tmp_df[-1][j] = tmp_df[-1][j] > threshold
  tmp = tmp_df[0].astype(int)+tmp_df[1].astype(int)+tmp_df[2].astype(int)+tmp_df[3].astype(int)+tmp_df[4].astype(int)



  return_logits = []
  for i in range(5):
    print(i)
    test_polarity_classification_model_path = polarity_classification_model_path +'saved_model_epoch_{}.pt'.format(i)

    polarity_model = RoBertaBaseClassifier_polar(len(polarity_id_to_name), len(tokenizer))
    polarity_model.load_state_dict(torch.load(test_polarity_classification_model_path, map_location=device))
    polarity_model.to(device)


    tmp_input_ids = []
    tmp_attention_masks = []
    tmp_labels = []
    logits = []
    tmp_logits = []


    for j in range(len(test_data)):
      tmp_annotation = []
      if tmp.iloc[j,:].max()>=3:
        for k in list(tmp):
          if tmp.loc[j,k]>=3:
            tokenized_data = tokenizer(test_data[j]['sentence_form'], k, padding='max_length', max_length=max_len, truncation=True)
            tmp_input_ids.append(tokenized_data['input_ids'])
            tmp_attention_masks.append(tokenized_data['attention_mask'])

            if len(tmp_input_ids) ==1000:
              input_ids = torch.tensor(tmp_input_ids).to(device)
              attention_masks = torch.tensor(tmp_attention_masks).to(device)
              with torch.no_grad():
                _, ce_logits = polarity_model(input_ids, attention_masks, None)
              if len(logits) == 0:
                logits = copy.deepcopy(ce_logits)
                tmp_input_ids = []
                tmp_attention_masks = []
              else:
                logits = torch.cat([logits, copy.deepcopy(ce_logits)], dim=0)
                tmp_input_ids = []
                tmp_attention_masks = []

      else:
        k = list(tmp)[np.argmax(entity_property_logits_mean[j])]
        # list(tmp)[np.argmax(tmp.iloc[j,:])]
        tokenized_data = tokenizer(test_data[j]['sentence_form'], k, padding='max_length', max_length=max_len, truncation=True)
        tmp_input_ids.append(tokenized_data['input_ids'])
        tmp_attention_masks.append(tokenized_data['attention_mask'])

        if len(tmp_input_ids) ==1000:
          input_ids = torch.tensor(tmp_input_ids).to(device)
          attention_masks = torch.tensor(tmp_attention_masks).to(device)
          with torch.no_grad():
            _, ce_logits = polarity_model(input_ids, attention_masks, None)
          if len(logits) == 0:
            logits = copy.deepcopy(ce_logits)
            tmp_input_ids = []
            tmp_attention_masks = []
          else:
            logits = torch.cat([logits, copy.deepcopy(ce_logits)], dim=0)
            tmp_input_ids = []
            tmp_attention_masks = []


    if len(tmp_input_ids) != 0:
      input_ids = torch.tensor(tmp_input_ids).to(device)
      attention_masks = torch.tensor(tmp_attention_masks).to(device)
      with torch.no_grad():
        _, ce_logits = polarity_model(input_ids, attention_masks, None)
      if len(logits) == 0:
        logits = copy.deepcopy(ce_logits)
      else:
        logits = torch.cat([logits, copy.deepcopy(ce_logits)], dim=0)

    return_logits.append(logits)
    del polarity_model
  return return_logits


In [ ]:

with open(category_extraction_model_path +'entity_property_logits.pkl', 'rb') as f:
    entity_property_logits = pickle.load(f)

test_data = jsonlload(test_data_path)
with open(category_extraction_model_path +'f1_scores.pkl', 'rb') as f:
    f1_scores = pickle.load(f)
logits = predict_polarity_vote(test_data, entity_property_logits, f1_scores)
with open(polarity_classification_model_path+'polarity_logits_vote.pkl', 'wb') as f:
  pickle.dump(logits, f, pickle.HIGHEST_PROTOCOL)

In [ ]:

with open(category_extraction_model_path +'entity_property_logits.pkl', 'rb') as f:
    entity_property_logits = pickle.load(f)
    

In [ ]:
import pickle

test_data = jsonlload(test_data_path)

with open(category_extraction_model_path +'f1_scores.pkl', 'rb') as f:
    f1_scores = pickle.load(f)

In [ ]:
import pickle
with open(category_extraction_model_path+'entity_property_logits.pkl', 'rb') as f:
    entity_property_logits = pickle.load(f)
entity_property_logits

In [ ]:
def make_annotation_vote(test_data, entity_property_logits, polarity_logits, f1_scores):
  polarity_logits = (polarity_logits[0]+polarity_logits[1]+polarity_logits[2]+polarity_logits[3]+polarity_logits[4])/5
  entity_property_logits_mean = entity_property_logits[0]+entity_property_logits[1]+entity_property_logits[2]+entity_property_logits[3]+entity_property_logits[4]
  entity_property_logits_mean = np.asarray((entity_property_logits_mean/5).sigmoid())
  tmp_df = []
  for i in range(len(f1_scores)):
    tmp_df.append(pd.DataFrame(np.asarray(entity_property_logits[i].sigmoid()), columns = entity_property_pair))
    for j in list(f1_scores[i]):
      f1_scores[i][j]
      threshold = np.arange(30, 95, 1)[np.argmax(f1_scores[i][j])]/100 
      tmp_df[-1][j] = tmp_df[-1][j] > threshold
  tmp = tmp_df[0].astype(int)+tmp_df[1].astype(int)+tmp_df[2].astype(int)+tmp_df[3].astype(int)+tmp_df[4].astype(int)
  i=0
  for j in range(len(test_data)):
    tmp_annotation = []
    if tmp.iloc[j,:].max()>=3:
      for k in list(tmp):
        if tmp.loc[j,k]>=3:


          ce_predictions = torch.argmax(polarity_logits[i], dim = -1)
          i += 1

          ce_result = polarity_id_to_name[ce_predictions.item()]
          tmp_annotation.append((k, ce_result))
    else:
      # k = np.argmax(tmp.iloc[j,:])
      k = np.argmax(entity_property_logits_mean[j])

      ce_predictions = torch.argmax(polarity_logits[i], dim = -1)
      i += 1

      ce_result = polarity_id_to_name[ce_predictions.item()]
      tmp_annotation.append((tf_id_to_name[k], ce_result))
      
    test_data[j]['annotation'] = tmp_annotation


  return test_data

In [ ]:
import pickle
with open(category_extraction_model_path+'entity_property_logits.pkl', 'rb') as f:
    entity_property_logits = pickle.load(f)

test_data = jsonlload(test_data_path)

with open(category_extraction_model_path +'f1_scores.pkl', 'rb') as f:
    f1_scores = pickle.load(f)

with open(polarity_classification_model_path+'polarity_logits_vote.pkl', 'rb') as f:
    logits = pickle.load(f)

In [ ]:
test_data2 = make_annotation_vote(test_data, entity_property_logits, logits, f1_scores)

with open(polarity_classification_model_path+'test_data2_vote.pkl', 'wb') as f:
  pickle.dump(test_data2, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

In [ ]:
import pickle
with open(polarity_classification_model_path+'test_data2_vote.pkl', 'rb') as f:
    test_data2 = pickle.load(f)
jsondump(test_data2, polarity_classification_model_path+'predict_result_vote.json')


In [ ]:
jsonlload(polarity_classification_model_path+'predict_result_vote.json')

In [ ]:
test_data2 = jsonlload(polarity_classification_model_path+'predict_result_vote.json')[0]
for i in range(len(test_data2)):
  if len(test_data2[i]['annotation'])>1:
    print(test_data2[i])